[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/r-xue/casa_proc/blob/master/demo/test_casaproc.ipynb)

## Example BX610: perform model fitting using method='emcee'

In [5]:
import sys
import glob
import os
import io
import logging
import emcee
from pprint import pprint

print(sys.version)


import socket 
if  'hypersion' or 'mini' in socket.gethostname() :
    os.chdir('/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/output/')
print(socket.gethostname())
print(os.getcwd())

import gmake
print(gmake.__version__)
print(gmake.__email__)
print(gmake.__demo__)
gmake.check_deps()

inpfile=gmake.__demo__+'/../examples/inpfile/bx610_band6_uv_mc.inp'
logfile=''

print('>'*40)
#gmake.logger_config()
#gmake.logger_status()
#import pprint
#pprint.pprint(logging.Logger.manager.loggerDict) 
gmake.logger_config()
inp_dct=gmake.read_inp(inpfile)
outdir=inp_dct['general']['outdir']
gmake.logger_config(logfile=outdir+'/gmake.log',loglevel='DEBUG',logfilelevel='DEBUG')
gmake.logger_status()


astropy            >=3.2.2      3.2.2       
emcee              >=3.0.0      3.0.0       
corner             >=2.0        2.0.1       
tqdm               unspecified  4.31.1      
lmfit              unspecified  0.9.12      
asteval            >=0.9.14     0.9.14      
numexpr            >=2.7.0      2.7.0       
hickle             unspecified  3.4.5       
alpy               unspecified  0.22.0      
regions            unspecified  0.3         
scipy              unspecified  1.2.1       
reproject          unspecified  0.6.dev644  
python-casacore    >=3.1.1      3.1.1       
scikit-image       unspecified  0.14.2      
galpy              unspecified  1.5.dev0    
mkl-fft            unspecified  1.0.14      
pvextractor        >=0.2.dev327 0.2.dev327  
spectral-cube      >=0.4.5.dev  0.4.5.dev2235
radio-beam         >=0.3        0.3.3.dev397
reproject          >=0.6.dev    0.6.dev644  
casa-proc          unspecified  0.1.dev3    


3.7.5 (default, Oct 16 2019, 09:38:04) 
[Clang 11.0.0 (clang-1100.0.33.8)]
hyperion
/Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/output
0.2.dev1
rx.astro@gmail.com
/Users/Rui/Dropbox/Worklib/projects/GMaKE/gmake
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<Logger gmake (DEBUG)>
[<FileHandler /Users/Rui/Dropbox/Worklib/projects/GMaKE/examples/output/bx610_band6_uv_mc/gmake.log (DEBUG)>, <StreamHandler stderr (DEBUG)>]


In [19]:
dat_dct=gmake.read_data(inp_dct,fill_mask=True,fill_error=True)

read data (may take some time..)

Read: ../data/bx610/alma/2015.1.00250.S/bb2.ms

data@../data/bx610/alma/2015.1.00250.S/bb2.ms                (56354, 238)         102 MiB             
uvw@../data/bx610/alma/2015.1.00250.S/bb2.ms                 (56354, 3)           661 KiB             
weight@../data/bx610/alma/2015.1.00250.S/bb2.ms              (56354,)             220 KiB             161.43835
chanfreq@../data/bx610/alma/2015.1.00250.S/bb2.ms            (238,)                 250.7529 GHz   252.6043 GHz
chanfreq@../data/bx610/alma/2015.1.00250.S/bb2.ms            (238,)                   7.8118 MHz     7.8118 MHz
phasecenter@../data/bx610/alma/2015.1.00250.S/bb2.ms>>[356.53933333  12.82202778]


250.7528594380543 GHz 252.60426423403908 GHz
7.811834582214355 MHz 7.811834582214355 MHz



Read: ../data/bx610/alma/2015.1.00250.S/bb3.ms

data@../data/bx610/alma/2015.1.00250.S/bb3.ms                (53466, 238)          97 MiB             
uvw@../data/bx610/alma/2015.1.00250.S/bb3.ms                 (53466, 3)           627 KiB             
weight@../data/bx610/alma/2015.1.00250.S/bb3.ms              (53466,)             209 KiB             187.34055
chanfreq@../data/bx610/alma/2015.1.00250.S/bb3.ms            (238,)                 233.2543 GHz   235.1058 GHz
chanfreq@../data/bx610/alma/2015.1.00250.S/bb3.ms            (238,)                  -7.8118 MHz    -7.8118 MHz
phasecenter@../data/bx610/alma/2015.1.00250.S/bb3.ms>>[356.53933333  12.82202778]

Read: ../data/bx610/alma/2015.1.00250.S/bb1.mfs.ms

data@../data/bx610/alma/2015.1.00250.S/bb1.mfs.ms            (56354, 1)           440 KiB             
uvw@../data/bx610/alma/2015.1.00250.S/bb1.mfs.ms             (56354, 3)           661 KiB             
weight@../data/bx610/alma/2015.1.00250.S/bb1.mfs.ms          (56354,

233.25434997369425 GHz 235.10575476967907 GHz
-7.811834582214355 MHz -7.811834582214355 MHz
250.07337356705435 GHz 250.07337356705435 GHz
1.859216630559723 GHz 1.859216630559723 GHz
235.87990757677653 GHz 235.87990757677653 GHz
-1.8592166305670168 GHz -1.8592166305670168 GHz


In [14]:
gmake.gmake_utils.get_obj_size(dat_dct)/1024/1024

203.66967391967773

In [6]:
from gmake import model_lnprob
lnl,blobs=model_lnprob(fit_dct['p_start'],fit_dct,inp_dct,dat_dct,
                               savemodel='')
#fit_dct,sampler=gmake.fit_setup(inp_dct,dat_dct)
print(type(blobs))
print(np.shape(blobs))

NameError: name 'fit_dct' is not defined

In [ ]:
blobs.item()['npar']

In [ ]:
gmake.fit_iterate(fit_dct,sampler,nstep=inp_dct['optimize']['niter'])

In [ ]:
gmake.fit_analyze(inpfile)